In [25]:
# Set directories 
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)
data_dir = os.path.join(os.getcwd(), '..', 'data')

# Import Functions
import pandas as pd
from d00_utils.example_func import greet



In [6]:
order_product_prior = pd.read_csv(os.path.join(data_dir,"01_raw/order_products__prior.csv"))
order_product_train = pd.read_csv(os.path.join(data_dir,"01_raw/order_products__train.csv"))
orders = pd.read_csv(os.path.join(data_dir,"01_raw/orders.csv"))
products = pd.read_csv(os.path.join(data_dir,"01_raw/products.csv"))
aisles = pd.read_csv(os.path.join(data_dir,"01_raw/aisles.csv"))
departments = pd.read_csv(os.path.join(data_dir,"01_raw/departments.csv"))


The goal of the competition is to predict whether a customer will repurchase a product that the specific customer already previously purchased. 

In [7]:
orders.groupby('eval_set').size()

eval_set
prior    3214874
test       75000
train     131209
dtype: int64

In [9]:
order_product_train.head()


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


The order_product_train contains for an order id the information which products were purchased in the order

What does the reordered 0 mean?:
* The items have not been (re)purchased in the train order, but were already purchased previously 
* are bought the first time in the train order and have never been purchased before 

If it’s the second case, then the zeros are missing in our train set. 

In [10]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


The orders df contains more info for each order. For example

* who purchased the order (order_id)
* to which set the orders belong (eval_set)


In [11]:
order_product_train.loc[order_product_train['order_id'] == 36, :]

,order_id,product_id,add_to_cart_order,reordered
8,36,39612,1,0
9,36,19660,2,1
10,36,49235,3,0
11,36,43086,4,1
12,36,46620,5,1
13,36,34497,6,1
14,36,48679,7,1
15,36,46979,8,1


The user 79431 has an order in the train set. The order_nr is 36 and its the users 23th order. Were the items 39612 and 49235 already previously purchased by the user, or are they purchased the first time in order 36?

Check if I correctly understand the data: 



In [32]:
order_ids_customer_79431 = orders.loc[orders['user_id'] == 79431, ['order_id','order_number']]
order_ids_customer_79431.head()

,order_id,order_number
1322233,396772,1
1322234,2968113,2
1322235,3080557,3
1322236,3260164,4
1322237,5239,5


In [33]:
order_product_joined = order_product_prior.merge(order_ids_customer_79431,how='inner',on = 'order_id' )
order_product_joined.head()


,order_id,product_id,add_to_cart_order,reordered,order_number
0,5239,19660,1,1,5
1,5239,38293,2,0,5
2,5239,21616,3,0,5
3,5239,43086,4,0,5
4,5239,19048,5,0,5


In [39]:
order_product_joined.loc[order_product_joined['product_id'] ==  49235,:]



,order_id,product_id,add_to_cart_order,reordered,order_number


In [40]:
order_product_joined.loc[order_product_joined['product_id'] ==  39612,:]


,order_id,product_id,add_to_cart_order,reordered,order_number


In [41]:
order_product_joined.loc[order_product_joined['product_id'] ==  46620,:]


,order_id,product_id,add_to_cart_order,reordered,order_number
152,2969365,46620,5,0,15


So we can answer the previously asked question. Orders in the train set where the variable repurchased = 0 have never been purchased by the customer. If the variable equals 1 then these items have been purchased previously. 

For the training data that means the following:
if repurchased = 0 then drop the row
All the prior orders have to be aggregated by user and each item which has been purchased previously before this specific train order have to get a 0 in the targt variable 



In [46]:
train_df = order_product_train.loc[order_product_train['reordered'] == 0,:]

#train_df.loc[:,'target'] = 1

/Users/matthiaslenfers/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/matthiaslenfers/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
